# Deep Learning for Robotics Week 01: 
## Predictive Model for Collision Avoidance

### Tutorial

Wider and more powerfull intelligence is necessary for the future of robotics. Currently, robots -- while capable of many things -- are often static, pre-programmed, and only operational in specificaly tailored environments. The vision for the future are dynamic robots capable of intelligently operating in changing environments. Machine learning -- which includes deep learning among other sub-fields -- provides a highly promising path forward. However, robots are a unique implementation of machine learning with many complications.

Many of the more common challenges in machine learning apply to robotics as well. For example: time, resources, and data are all still needed for the learning process, and deployment can provide many unexpected problems. There are also some complications in robotics that are somewhat unique. Safety is a huge concern, so any failure in a robotic system must be handled appropriately with potentialy dire concequences. Furthermore, robots often have limited processing capabilites, so latency and resource availability are constraining. This is further problematic due to the fact that robots are operating in real-time and need to make immediate decisions. These are only some of the challenges, but it certainly is a hard problem.

If these difficulties can be overcome then there is a broad array of applications for machine learning. To name a few broad classes: controls, state estimation, perception, coordination, modular systems, and many more. A classic example is computer vision which can (and does) allow robots to detect objects like people -- useful for human robot interaction scenarios. Just about any aspect of robotics could (and currently does) benefit form machine learning.

The goal of this tutorial is to train a network for collision detection and deploy this in a robotic simulation. The input to this network are distance sensor values -- which could be a sonic sensor, LiDAR, etc. -- and the steering angle. The output is a binary indicator of collision or no collision in the next time step. Without the steering angle the model could sometimes incorrectly predict as certain collision scenarios are dependent an the action taken.

The robot that will be used for this tutorial has 5 sensors spread across its front and a bumper sensor to detect an actual collision. Training data is collected by having the robot randomly explore a simulation environment.

### Code (Description and Changes)

The tutorial provides a large amount of code to be used in the collision avoidance exercise. We will describe the aspects of this code and any changes that we have made to it. Notably, the tutorial uses PyTorch as its deep learning library, but we will be converting everything into Tensorflow for a deeper learning experience.

**ExploreAndCollect.py:** This python file loads the simulator and has the robot move throughout the envrionment randomly. The sensor data is collected and stored in the "SensorData" folder for use in training the network later on.

**PreProcessing.py:** This python file loads the sensor data and preforms a few basic pre-processing tasks. Namely: it labels the data, duplicates the collision data, and then divides it into training and test sets.

**MakeItLearn.py:** This file use loads the saved sensor data, and then uses PyTorch to build and train a basic dense network. This model is then saved.

**PlayingTheModel.py:** This file loads the simulator, loads the network model, and has the robot begin travelling to a pre-defined destination. Along the way the robot uses the network to check for collisions, and if detected take alternative action.

We will be leaving **ExploreAndCollect** mostly unchanged (excepting a few simulation details), but will avoid **PreProcessing** and **MakeItLearn**. Instead this jupyter notebook will handle the details of building a network in Tensorflow, training it, and running the final simulation.

### Exercise

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
#Global vars
num_epochs = 20

In [3]:
#Load and preprocess the training data.
sensor_data = np.loadtxt('./SensorData/SensorData.txt')
print(sensor_data.shape)

(8001, 7)


There are 8,001 data each with 7 dimensions. The first 5 rows are the distance sensors, then the steering angle, and finally a binary indicator for collision. We don't just want the robot to predict a collision right as it collides, we want it to do so enough beforehand to change its direction. The next step is to mark data before the collision as a collision.

In [4]:
SensorDataRows = []
for i in range(20):
    SensorDataRows.append(np.roll(sensor_data[:,-1],-i-1))
for i in range(20):
    sensor_data[:,-1] += SensorDataRows[i]
    
print(max(sensor_data[:,-1]))

1.0


Becuase the data was gathered from the robot randomly moving around in a somewhat sparse environment, we may not have that much collision data. To fix this we will duplicate our collision data.

In [5]:
collision_data = sensor_data[sensor_data[:,-1] > 0]

#Duplicate 10 times
for i in range(10):
    sensor_data = np.append(sensor_data, collision_data, axis=0)
    
print(sensor_data.shape)

(18921, 7)


We can see we have increased the total data by a roughly 10000. The last step is to split our data into training and testing data.

In [6]:
np.random.shuffle(sensor_data) #Mix the data around

split = int(0.70*sensor_data.shape[0])

train_data = tf.constant(sensor_data[:split,:-1])
train_labels = tf.constant(sensor_data[:split,-1])

test_data = tf.constant(sensor_data[split:,:-1])
test_labels = tf.constant(sensor_data[split:,-1])

print(train_data.shape)
print(test_data.shape)

(13244, 6)
(5677, 6)


We will build a simple linear network for our collision prediciton.

In [7]:
#Build Network

model = keras.Sequential([
    keras.Input(shape=6),
    keras.layers.Dense(100, activation='tanh'),
    keras.layers.Dense(50, activation='tanh'),
    keras.layers.Dense(1),
])

model.compile(optimizer='adam',
             loss=keras.losses.MeanSquaredError(),
             metrics=['accuracy'])

# model.summary()

In [8]:
model.fit(train_data, train_labels, epochs=num_epochs)

Epoch 1/20
414/414 [==============================] - 0s 540us/step - loss: 0.0452 - accuracy: 0.9571
Epoch 2/20
414/414 [==============================] - 0s 533us/step - loss: 0.0174 - accuracy: 0.9870
Epoch 3/20
414/414 [==============================] - 0s 528us/step - loss: 0.0148 - accuracy: 0.9887
Epoch 4/20
414/414 [==============================] - 0s 570us/step - loss: 0.0140 - accuracy: 0.9890
Epoch 5/20
414/414 [==============================] - 0s 540us/step - loss: 0.0131 - accuracy: 0.9890
Epoch 6/20
414/414 [==============================] - 0s 543us/step - loss: 0.0128 - accuracy: 0.9894
Epoch 7/20
414/414 [==============================] - 0s 582us/step - loss: 0.0111 - accuracy: 0.9907
Epoch 8/20
414/414 [==============================] - 0s 548us/step - loss: 0.0114 - accuracy: 0.9903
Epoch 9/20
414/414 [==============================] - 0s 548us/step - loss: 0.0107 - accuracy: 0.9912
Epoch 10/20
414/414 [==============================] - 0s 564us/step - loss: 0.010

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               700       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 5,801
Trainable params: 5,801
Non-trainable params: 0
_________________________________________________________________


In [10]:
idx = 10
print(test_labels[idx])
print(test_data[idx])

test = np.expand_dims(test_data[idx],0)

pred = model(test, training=False)
print(pred)

tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor([100.         79.         60.         67.         28.        276.3692078], shape=(6,), dtype=float64)
tf.Tensor([[0.94886917]], shape=(1, 1), dtype=float32)


Due to a weird bug with Pygame and Jupyter combination, you can only run teh cell below once, and then you need to reset the kernel.

In [12]:
#Running the simulation
from PlayingTheModel import runSim
#loc can be any starting location of 1,2,3,4
runSim(model, loc=4)

[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100.  97. 100.]
[100. 100. 100.  90. 100.]
[100. 100. 100.  84. 100.]
[100. 100. 100.  77. 100.]
[100. 100. 100.  70. 100.]
[100. 100. 100.  64. 100.]
[100. 100. 100.  57. 100.]
[100. 100. 100.  51.  44.]
[100. 100. 100.  44.  38.]
[100. 100. 100.  38.  31.]
[100. 100. 100.  31.  25.]
[100. 100. 100.  33.  21.]
[100. 100. 100.  37.  19.]
[100. 100. 100.  43.  18.]
[100. 100. 100.  53.  18.]
[100. 100. 100.  67.  20.]
[100. 100. 100.  88.  23.]
[100. 100. 100. 100.  28.]
[100. 100. 100. 100.  36.]
[100. 100. 100. 100.  46.]
[100. 100. 100. 100.  60.]
[100. 100. 100. 100.  79.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[100. 100. 100. 100. 100.]
[

SystemExit: 